In [ ]:
#| hide
from learn_nanogpt.core import *

# learn_nanogpt

> Notes on Andrej NanoGpt

Read and inspect the tiny shakespeare data.

In [ ]:
with open("../data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(f"{len(text):_}")

1_115_394


First 1000 characters:

In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Get all unique characters:

In [ ]:
chars= sorted(list(set(text)))
vocab_size=len(chars)
print(vocab_size, ''.join(chars))

65 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


Create the encoder and decoders:

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

[46, 47, 1, 58, 46, 43, 56, 43]

In [ ]:
print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


Convert data to tensors:

In [ ]:
#| hide
from fastai.vision.all import *

/Users/JOSHIPX22/opt/anaconda3/envs/creon/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/JOSHIPX22/opt/anaconda3/envs/creon/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  Referenced from: <6E6BE615-472A-3225-994B-C7BC27D09EAE> /Users/JOSHIPX22/opt/anaconda3/envs/creon/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <9FBB2DD5-CEC1-358C-B1EF-43861AD5B797> /Users/JOSHIPX22/opt/anaconda3/envs/creon/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
data

tensor([18, 47, 56,  ..., 45,  8,  0])

In [ ]:
print(decode([i.item() for i in data[:100]]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


Prepare train and validation sets.

In [ ]:
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]
train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

`block_size` is the length of text on which transformer is trained. It is also called the context length.

In [ ]:
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(len(x)):
    context=x[:t+1]
    target=y[t]
    print(f"For context {context} target is {target}")

For context tensor([18]) target is 47
For context tensor([18, 47]) target is 56
For context tensor([18, 47, 56]) target is 57
For context tensor([18, 47, 56, 57]) target is 58
For context tensor([18, 47, 56, 57, 58]) target is 1
For context tensor([18, 47, 56, 57, 58,  1]) target is 15
For context tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
For context tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58
